# 4 决策树算法

## 4.1 算法描述：
采用基本的分而治之的思想，将一个大决定，分成若干小决定，根据每个小决定进行分类，然后得出最终决定。
算法描述


*输入*: 

   训练集 $D = \{(x_1,y_1),(x_2,y_2),\dots\,(x_m,y_m)\}$
     
   属性集 $A = \{a_1,a_2, \dots, a_d\}$

*过程*:
```python
def TreeGenerate(D,A):  
    生成节点node；
    if D中样本全属于同一类别C:
        将node标记为C类叶节点
        return
    if A == Ø or D中样本在A上取值相同:
        将node标记为叶节点，其类别标记为D中样本数最多的类
        return
    从A中选择最优划分属性av
    for a in av:
        为node的生成一个分支，令Dv表示D在av上取值为a的样本子集
        if Dv == Ø:
            将分支节点标记为叶节点，其类别标记为D中样本最多的类
            return
        else:
            以TreeGenerate(Dv, A\{a})为分支节点
```
*输出*:以node为一根节点的一颗决策树

## 4.2 划分选择
由算法我们可以看出，决策树学习的关键是“选择最优划分属性”。一般而言，随着划分过程不断进行，我们希望决策树的分支节点所包含的样本尽可能属于同一类别，即节点的"纯度"越来越高。
### 4.2.1 信息增益
   “信息熵”(information entropy)是度量样本集合纯度最常用的一种指标，假定当前样本集合D中第k类样本所占比例为$p_k(k=1,2,\dots\,\vert\nu\vert)$则D的信息熵定义为
   $$Ent(D) =-\sum_{k=1}^{\vert\nu\vert}p_klog_2p_k$$
Ent(D)值越小,D的纯度越高。
假定离散属性a有V个可能的取值$\{a^1,a^2,\dots\,a^{\nu}\}$，若使用a对样本集D进行划分，则会产生V个分支节点，其中第$v$个包含了D中所有在属性a上取值为$a^{\nu}$的样本，记为$D^{\nu}$,我们可以计算出$D^{\nu}$的信息熵，在考虑到不同的分支节点所包含的样本数不同，给分支节点赋予权重$\vert{D^{\nu}}\vert/\vert{D}\vert$,即样本数越多的分支节点的影响越大，于是可计算出用属性a对样本集D进行划分所得“信息增益”(information gain)
$$Gain(D, a)= Ent(D) =  \sum_{v=1}^{V} \frac{\vert{D^{\nu}}\vert}{\vert{D}\vert}{Ent(D^{\nu})}$$
一般而言信息增益越大，则意味着使用属性a来进行划分所获得的“纯度提升”越大。著名的ID3决策树学习法就是以信息增益为准则来选择划分属性。


### 示例
西瓜数据集


In [263]:
import pandas as pd
import numpy as np
from cytoolz import *
from itertools import product
from operator import *

melon = pd.DataFrame({'色泽':['青绿', '乌黑', '乌黑', '青绿', '浅白','青绿', '乌黑', '乌黑', '乌黑', '青绿', '浅白', '浅白', '青绿','浅白', '乌黑', '浅白','青绿'],
 '根蒂':['蜷缩', '蜷缩', '蜷缩', '蜷缩', '蜷缩','稍蜷', '稍蜷', '稍蜷', '稍蜷', '硬挺', '硬挺', '蜷缩', '稍蜷', '稍蜷', '稍蜷','蜷缩','蜷缩'],
 '敲声':['浊响', '沉闷', '浊响', '沉闷', '浊响','浊响', '浊响', '浊响', '沉闷', '清脆', '清脆', '浊响', '浊响', '沉闷', '浊响','浊响','沉闷'],
 '纹理':['清晰', '清晰', '清晰', '清晰', '清晰','清晰', '稍糊', '清晰', '稍糊', '清晰', '模糊', '模糊', '稍糊', '稍糊', '清晰','模糊','稍糊'],
 '脐部':['凹陷', '凹陷', '凹陷', '凹陷', '凹陷','稍凹', '稍凹', '稍凹', '稍凹', '平坦', '平坦', '平坦', '凹陷', '凹陷', '稍凹','平坦','稍凹'],
 '触感':['硬滑', '硬滑', '硬滑', '硬滑', '硬滑','软粘', '软粘', '硬滑', '硬滑', '软粘', '硬滑', '软粘', '硬滑', '硬滑', '软粘','硬滑','硬滑'],
 '好瓜':['是', '是', '是', '是', '是','是', '是', '是', '否', '否', '否', '否', '否', '否', '否','否','否']})

def EntropyBase(x):
    if x == 0:
        return 0;
    else:
        return x*np.log2(x)

def Entropy(p):
    return reduce(add, map(lambda x: -EntropyBase(x), p))

def Ent(df, y):
    p = list(map(lambda x: len(df[df[y]==x])/len(df), np.unique(df[y])))
    return Entropy(p)
    
def EntSub(df, col,y):
    sz = df.groupby([col, y])[y].count()
    def g(sz, c, t):
        try:
            return sz[c, t]
        except:
            return 0
    ens = np.array(list(map(lambda c: list(map(lambda t: g(sz,c,t),np.unique(df[y]))), np.unique(df[col]))))
    return list(map(lambda x : Entropy(x/x.sum()), ens))

def Dv(df, col, y):
    return df.groupby([df[col]])[y].count().values

def Gain(df, col, y):
    nm = Dv(melon, col, y)
    es = EntSub(melon, col, y)
    n = sum(nm)
    return Ent(df, y) - reduce(add, map(lambda x: x[0]*x[1]/n, zip(es, nm)))
    
def IV(df, col, y):
    n = len(df)
    return reduce(add, map(lambda x: -EntropyBase(x/n), Dv(df, col, y)))    

In [264]:
IV(melon, '触感', '好瓜')

0.8739810481273578

In [260]:
gains = pd.DataFrame({'value':list(map(lambda x: Gain(melon, x,'好瓜'), melon.columns.values[1:])),
 'name':melon.columns.values[1:]})
gains

,name,value
0,敲声,0.140781
1,根蒂,0.142675
2,纹理,0.380592
3,脐部,0.289159
4,色泽,0.108125
5,触感,0.006046


In [238]:
Gain(melon, '色泽', '好瓜')

0.1081251652653652

In [223]:
EntSub(melon, '色泽', '好瓜')

[0.9182958340544896, 0.7219280948873623, 1.0]

In [224]:
Dv(melon, '色泽', '好瓜')

array([6, 5, 6])

In [225]:
Ent(melon, '好瓜')

0.9975025463691152

### 4.2.2 增益率
实际上信息增益对可取值数目较多的属性有所偏好，为减少这种偏好带来的不利影响，著名的C4.5决策树算法不直接使用信息增益，而使用增益率(gain radio)来选择最优划分属性，增益率定义如下$$Gain\_ratio(D, \alpha) = \frac{Gain(D,\alpha)}{IV(\alpha)}$$
其中$$IV(\alpha)=-\sum^{V}_{v=1}\frac{\vert{D}^v\vert}{\vert{D}\vert}log_2\frac{\vert{D}^v\vert}{\vert{D}\vert}$$

In [271]:
ivs = pd.DataFrame({'value':list(map(lambda x: IV(melon, x,'好瓜'), melon.columns.values[1:])),
 'name':melon.columns.values[1:],
 'count':list(map(lambda x: len(np.unique(melon[x])), melon.columns.values[1:]))})
ivs

,count,name,value
0,3,敲声,1.332820
1,3,根蒂,1.402081
2,3,纹理,1.446648
3,3,脐部,1.548565
4,3,色泽,1.579863
5,2,触感,0.873981


增益率准则可以对取值数目较少的属性有所偏好，因此C4.5算法并不是直接选择增益率最大的候选划分属性，而是使用了一个启发式，先从候选划分属性中找出信息增益率高于平均水平的属性，在从中选取增益率最高的。

### 4.2.3 基尼指数
CART决策树使用基尼指数(Gini index)来选择划分属性，数据集D的纯度可以用基尼值来度量
$$Gini(D)=\sum_{k=1}^{\vert{Y}\vert}\sum_{k^{'}\not=k}{p_kp_{k^{'}}}$$
$$=1-\sum_{k=1}^{\vert{Y}\vert}p_{p}^{2}$$
直观上来说，Gini(D)反映了从数据集D中随机抽取两个样本，其类别标记不一致的概率，因此Gini(D)越小，则数据集D的纯度越高。

属性a的基尼指数定义为$$Gini\_index(D, \alpha)=\sum_{v=1}^{V}\frac{\vert{D^v}\vert}{\vert{D}\vert}Gini(D^{v})$$

## 4.3 剪枝处理
### 4.3.1 预剪枝 prepruning
在决策树的生成过程中，对每个节点在划分前进行评估，若当前节点的划分不能带来决策树泛型化性能提升，则停止划分并将当前节点标记为叶节点。
### 4.3.2 后剪枝 postpruning
先从训练集生成一颗完整的决策树，然后自底向上对非叶节点进行考察，若将该结点对应的子树替换为叶节点，能带来决策树泛化性能提升，则将该子节点替换为叶节点。



## 4.4连续与缺失值
### 4.4.1 连续值处理
由于连续属性的可取值数目不再有限，因此，不能直接根据连续属性的可取值来对节点进行划分，此时，连续属性离散化技术可派上用场。最简单的策略是二分法(bi-partition)对连续属性进行处理，这正是C4.5决策树算法采用的机制。
给定样本集D和连续属性a，假定a在D上出现了n个不同的取值，将这些值从小到大进行排序，记为$\{a^{1},a^{2},\dots\,a^{n}\}$.基于划分点t可将D分成子集$D_t^{-}$和$D_t^{+}$，其中$D_t^{-}$包含哪些在属性a上取值不大于t的样本，而$D_t^{+}$则包含哪些在属性a上取值大于t的样本。显然，对相邻的属性取值$a^{i}$和$a^{i+1}$来说，t在区间$[a^{i},a^{i+1})$中取任意值所产生的划分结果相同，因此对连续属性a我们可考察包含n-1个元素的候选划分点集合$$T_a=\left\{\frac{a^i+a^{i+1}}{2}\vert1 \leq i \leq n-1\right\}$$
即把区间$[a^{i},a^{i+1})$的中位点作为候选划分点，然后我们就可以像离散属性值一样来考察这些点，选取最优的划分点进行样本集合的划分。得出如下公式$$Gain(D, a) =  \mathop{max}_{t \in T_a} Gain(D,a,t)$$
$$\mathop{max}_{t\in T_a}Ent(D) - \sum_{\lambda\in\{-,+\}}\frac{\vert{D_t^{\lambda}}\vert}{\vert{D}\vert}Ent(D_t^{\lambda})$$
### 4.4.2 缺失值处理
现实任务中常会遇到不完整的样本，即样本的某些属性缺失，